<a href="https://colab.research.google.com/github/kollivinitha/Speech-to-Vision/blob/main/Speech_to_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install python3-pyaudio
!pip install SpeechRecognition
!pip install pydub


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pyaudio is already the newest version (0.2.11-1.3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
from pydub import AudioSegment
import speech_recognition as sr
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!pip install modelscope==1.4.2
!pip install open_clip_torch
!pip install pytorch-lightning

  Using cached open_clip_torch-2.24.0-py3-none-any.whl (1.5 MB)
  Using cached ftfy-6.2.0-py3-none-any.whl (54 kB)
  Using cached timm-0.9.16-py3-none-any.whl (2.2 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:0

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys

p = pipeline('text-to-video-synthesis', 'damo/text-to-video-synthesis')

2024-03-28 08:36:22,450 - modelscope - INFO - PyTorch version 2.2.1+cu121 Found.
2024-03-28 08:36:22,459 - modelscope - INFO - TensorFlow version 2.15.0 Found.
2024-03-28 08:36:22,464 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-03-28 08:36:22,468 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-03-28 08:36:22,588 - modelscope - INFO - Loading done! Current index file version is 1.4.2, with md5 22cbf0f526d8bb0dbe39c789fa93730f and a total number of 842 components indexed
2024-03-28 08:36:33,308 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.0
Downloading: 100%|██████████| 782k/782k [00:00<00:00, 2.78MB/s]
Downloading: 100%|██████████| 796k/796k [00:00<00:00, 2.36MB/s]
Downloading: 100%|██████████| 737k/737k [00:00<00:00, 2.62MB/s]
Downloading: 100%|██████████| 846k/846k [00:00<00:00, 3.03MB/s]
Downloading: 100%|██████████| 729k/729

In [ ]:
def convert_to_wav(input_file, output_file):
    audio = AudioSegment.from_ogg(input_file)
    audio.export(output_file, format="wav")


In [ ]:
# Function to convert audio file to text
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            print("Sorry, could not understand audio")
            return ""
        except sr.RequestError as e:
            print("Error fetching results; {0}".format(e))
            return ""

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    porter_stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    lemmatized_text = ' '.join(lemmatized_tokens)

    return lemmatized_text

In [ ]:
# Main function
def main():
    # Input and output file paths
    input_file = "/content/drive/MyDrive/PTT-20240328-WA0050.opus"
    output_file = "/content/drive/MyDrive/converted_audio.wav"

    # Convert .ogg to .wav
    convert_to_wav(input_file, output_file)

    # Convert audio to text
    text = speech_to_text(output_file)
    print("Text from audio:", text)

    # Preprocess text
    preprocessed_text = preprocess_text(text)
    print("Preprocessed text:", preprocessed_text)

    test_text = {
        'text': preprocessed_text,
    }
    output_video_path = p(test_text,)[OutputKeys.OUTPUT_VIDEO]
    print('output_video_path:', output_video_path)
    from google.colab import files
    files.download(output_video_path)


if __name__ == "__main__":
    main()

2024-03-28 09:56:01,851 - modelscope - WARNING - task text-to-video-synthesis input definition is missing


Text from audio: a girl is writing
Preprocessed text: a girl is writing


2024-03-28 09:56:57,162 - modelscope - WARNING - task text-to-video-synthesis output keys are missing


output_video_path: /tmp/tmp9mgje19k.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>